In [2]:
import pandas as pd
import numpy as np
import plotnine as p9
import pyfpgrowth as fp

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import time

In [3]:
# Reading JSON data into python as pandas dataframe.

data = pd.read_json('/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION.json',lines=True)
meta_data = pd.read_json('/Users/abdulrehman/Desktop/usml/data/meta_AMAZON_FASHION.json',lines=True)


In [4]:
# Converting unix time to Date format and storing it in reviewTime Column.

data['reviewTime']=data.apply(lambda row: list(time.gmtime(row.unixReviewTime))[:-6], axis=1)
data = data.sort_values('unixReviewTime')
data['year']=data.apply(lambda row: row.reviewTime[0], axis=1)


## We will Make 2 different User item matrix.
   ### 1. With items with overall rating of 5 and above.
   ### 2. With all the items, including items with bad ratings.

### Method to create user itemsets.

In [12]:
def make_useritem_matrix(byreviewer):
    reviewers = list(byreviewer.reviewerID.unique())
    itemratedf = pd.DataFrame(data={'reviewerID':reviewers})
    for index, row in byreviewer.iterrows():
        itemratedf.loc[itemratedf['reviewerID'] == row['reviewerID'], row['asin']] = row['overall']
    return itemratedf

## 1. Creating data with items with overall rating of 5 and above.

In [7]:
# Now we will prepare data to make recommendations only based on reviews which have 4 or more ratings.
# We will delete all the rows with overall ratings less than 4.
# Then we remove all customers who have made less than 9 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

data.drop(data[ data['overall'] < 4 ].index , inplace=True)

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 9)

## Saving data and user itemset with items with overall rating of 5 and above.

In [11]:
itemratedf = make_useritem_matrix(byreviewer)
itemratedf.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_posonly_user_itemset.csv', index = False)
byreviewer.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_posonly_user_clean.csv', index = False)

## 2. Creating data with all the items, including items with bad ratings.

In [13]:
data = pd.read_json('/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION.json',lines=True)

# We remove all customers who have made less than 9 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 9)

## Saving data and user itemset with items with all the items, including items with bad ratings.

In [14]:
itemratedf = make_useritem_matrix(byreviewer)
itemratedf.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_user_itemset.csv', index = False)
byreviewer.to_csv(r'/Users/abdulrehman/Desktop/usml/data/AMAZON_FASHION_clean.csv', index = False)

In [ ]:
# We will now do EDA on the cleaned Data in EDA.ipynb